In [ ]:
# Azure Machine Learning ワークスペースを読み込む
import os
import tensorflow as tf
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')
print("TensorFlow version == {}".format(tf.__version__))

In [ ]:
# イメージの前処理
# ResNet 50 モデルへの入力となるテンソルとして、JPEG 画像を文字列とする 
import azureml.accel.models.utils as utils
tf.reset_default_graph()

in_images = tf.placeholder(tf.string)
image_tensors = utils.preprocess_array(in_images)
print(image_tensors.shape)

In [ ]:
# 特徴抽出器を読み込む
# 特徴抽出器として使用される ResNet50 の量子化されたバージョンの TensorFlow チェックポイントをダウンロード
from azureml.accel.models import QuantizedResnet50
save_path = os.path.expanduser('~/models')
model_graph = QuantizedResnet50(save_path, is_frozen=True)
feature_tensor = model_graph.import_graph_def(image_tensors)
print(model_graph.version)
print(feature_tensor.name)
print(feature_tensor.shape)

In [ ]:
# 分類子を追加する
# ImageNet データ セットでトレーニングされた分類子 classifier を追加 (ここでは再トレーニングは行わない)
classifier_output = model_graph.get_default_classifier(feature_tensor)
print(classifier_output)

In [ ]:
# モデルを保存する
# プリプロセッサ、ResNet 50 特徴抽出器、および分類子を読み込んだ後は、グラフと関連する変数をモデルとして保存
model_name = "resnet50"
model_save_path = os.path.join(save_path, model_name)
print("Saving model in {}".format(model_save_path))

with tf.Session() as sess:
    model_graph.restore_weights(sess)
    tf.saved_model.simple_save(sess, model_save_path,
                               inputs={'images': in_images},
                               outputs={'output_alias': classifier_output})

In [ ]:
# 入出力のテンソルを保存する
input_tensors = in_images.name
output_tensors = classifier_output.name

print(input_tensors)
print(output_tensors)

In [ ]:
# モデルの登録
# SDK と Azure Blob Storage 内の ZIP ファイルを使用して、モデルを登録
from azureml.core.model import Model

registered_model = Model.register(workspace=ws,
                                  model_path=model_save_path,
                                  model_name=model_name)

print("Successfully registered: ", registered_model.name,
      registered_model.description, registered_model.version, sep='\t')

In [ ]:
# モデルを変換する
# TensorFlow グラフを、Open Neural Network Exchange 形式 (ONNX) に変換
from azureml.accel import AccelOnnxConverter

convert_request = AccelOnnxConverter.convert_tf_model(
    ws, registered_model, input_tensors, output_tensors)

# If it fails, you can run wait_for_completion again with show_output=True.
convert_request.wait_for_completion(show_output=False)

# If the above call succeeded, get the converted model
converted_model = convert_request.result
print("\nSuccessfully converted: ", converted_model.name, converted_model.url, converted_model.version,
      converted_model.id, converted_model.created_time, '\n')

In [ ]:
# Docker イメージを作成する
# 変換されたモデルとすべての依存関係を Docker イメージに追加
from azureml.core.image import Image
from azureml.accel import AccelContainerImage

image_config = AccelContainerImage.image_configuration()
# Image name must be lowercase
image_name = "{}-image".format(model_name)

image = Image.create(name=image_name,
                     models=[converted_model],
                     image_config=image_config,
                     workspace=ws)
image.wait_for_creation(show_output=False)

In [ ]:
# タグ別にイメージを一覧表示し、デバッグ用に詳細ログを取得
for i in Image.list(workspace=ws):
    print('{}(v.{} [{}]) stored at {} with build log {}'.format(
        i.name, i.version, i.creation_state, i.image_location, i.image_build_log_uri))